In [1]:
import os

import numpy as np
import scipy.sparse as sps
from pyvis.network import Network
from particle import Particle
import vector

from cluster_gnn.data import internal as Data
from cluster_gnn.features import build_features as Features

In [14]:
ROOT_DIR = os.path.expanduser('~/projects/cluster_gnn/')

with Data.EventLoader(ROOT_DIR + 'data/processed/events.hdf5', 'wboson') as evts:
    # set data object state to point at first event
    evts.set_evt(5257)
    # form weighted edges
    pmu = evts.get_pmu()
    sps_adj = sps.coo_matrix(
        Features.knn_adj(
            Features.deltaR_aff(pmu),
            k=8,
            weighted=True,
            dtype=np.float32
            )
        )
    edge_idxs = zip(tuple(int(idx) for idx in sps_adj.row),
                    tuple(int(idx) for idx in sps_adj.col),
                    tuple(1.0 / (float(wt) + 0.1) for wt in sps_adj.data)
                    )
    # get data from particle ids
    pcls = [Particle.from_pdgid(pdg) for pdg in evts.get_pdg()]
    labels = list(map(lambda pcl: pcl.name, pcls))
    ids = list(range(len(pcls)))
    node_vals = Features._array_to_vec(pmu).pt
    node_vals = node_vals * 10.0 / node_vals.max()
    node_vals = tuple(float(val) for val in node_vals)
    # identify which are W boson
    groups = list(evts.get_signal(key='is_from_W').astype(np.uint8))
    
net = Network(height=600, width=800, notebook=True)
net.toggle_hide_edges_on_drag(False)
net.barnes_hut()
net.add_nodes(ids,
          label=labels,
          value=node_vals,
          color=['#162347' if group == 0 else '#dd4b39' for group in groups],
          )
net.add_edges(edge_idxs)
net.show('ex.html')